In [1]:
import numpy as np
import scipy.sparse as scpsp
import tqdm
from clutils import *
import pyopencl.array as ca
import os
os.environ["PYOPENCL_CTX"] = "0"
import matplotlib.pyplot as plt

In [2]:
for bs in [1,2,4,8,16,32,64,128,256]:
        print(f' n sims in parallel : {bs}') # = 32 # n sims in parallel
        gamma_range = np.linspace(0.001, 0.01, bs).astype('f')
        nsims = np.int32(len(gamma_range))
        
        # load the global and local connectivity matrices
        lc_dist_mat = scpsp.load_npz('vert2vert_gdist_mat_32k.npz').astype('f')
        gc_dist_mat = scpsp.load_npz('vert2vert_lengths_32k_15M.npz').astype('f')
        gc_mat = scpsp.load_npz('vert2vert_weights_32k_15M.npz').astype('f')

        # make lc kernel from gdist
        lc_mat = lc_dist_mat.copy()
        lc_mat.data = np.exp(-lc_mat.data/5.0).astype('f')
        nv = lc_mat.shape[0]

        # take subset for benchmarking
        nv = np.int32(10*1024)
        gc_mat = gc_mat[:nv][:,:nv]
        lc_mat = lc_mat[:nv][:,:nv]
        gc_dist_mat = gc_dist_mat[:nv][:,:nv]
        lc_dist_mat = lc_dist_mat[:nv][:,:nv]

        # some parameters
        dt = np.float32(0.1)

        # prepare extra info for delays
        local_velocity = 6 # # Salami et al. 2002, intra-cortical conduction 10x slower 
        v2v_velocity = 6 #3.9 # according to Lemarechal et al. 2022
        igc = (gc_dist_mat / local_velocity / dt).astype('i') # rounds down
        ilc = (lc_dist_mat / v2v_velocity / dt).astype('i')

        nh = np.max([igc.max(), ilc.max()])
        nh += 1 # add one to buffer 0 delay state
        # round up to the next power of 2
        nh = (2**np.ceil(np.log2(nh))).astype('i')      
        
        util = Util()
        util.load_kernel('./delays_sparse_buf.opencl.c', 'delays1')
        util.load_kernel('./delays_sparse_buf.opencl.c', 'upbuf_heavi')
        util.load_kernel('./heun_determ.opencl.c', 'heun_pred_determ', 'heun_corr_determ')
        util.load_kernel('./epi2d.opencl.c', 'epi2d_dfun')
        util.init_vectors('xi zi lc1 lc2 gc1 gc2 dx1 dx2 dz1 dz2', (nv * nsims,))

        # initialise sparse buffer
        # initialised with zeros, as if in the past the state was below the heaviside thershold "theta"
        util.init_vectors('xbuf', (nh * nv * nsims,)) 

        # initialise on fixed point
        x = np.ones((nv*nsims,),'f') * -1.337896365956364
        z = np.ones((nv*nsims,),'f') * -0.18514344764362828
        # but one vertex below fixed point to trigger one excitation
        z[:] = -0.3
        util.move(x=x, z=z)

        # set all vertices in excitable regime, won't oscillate on its own
        x_0 = np.ones((nv*nsims,),'f') * -1.291610504045457 
        util.move(x_0=x_0)

        # move connecitivty and delay informaiton to GPU
        util.move_csr(gc_mat=gc_mat, lc_mat=lc_mat)
        util.move(igc=igc.data,    ilc=ilc.data)

        # set coupling strength
        k_lc = np.zeros((nv*nsims,),'f')
        k_gc = np.tile(gamma_range, nv).astype('f')
        util.move(k_lc = k_lc, k_gc=k_gc)

        # local parameters
        I=np.float32(1.0)
        eps=np.float32(0.04)
        theta=np.float32(0.0)

        # handle boilerplate for kernel launches
        def do(f, *args):
                args = [(arg.data if hasattr(arg, 'data') else arg) for arg in args]
                f(util.queue, (nv * nsims,), (nsims,), *args)

        def step(t):
                # eval coupling 1 or 2, depending on time step
                if t % 2 == 0:
                        do(util.upbuf_heavi, nv, nsims, np.int32(t), theta, nh, util.xbuf, util.x)

                        # global coupling for heun corrector, i.e. t+1
                        do(util.delays1, 
                                nv, nsims, np.int32(t+1), nh, 
                                util.gc1, util.xbuf, 
                                util.gc_mat_data, util.igc, util.gc_mat_indices, util.gc_mat_indptr,)
                        
                        # local coupling for heun corrector, i.e. t+1
                        do(util.delays1, 
                                nv, nsims, np.int32(t+1), nh, 
                                util.lc1, util.xbuf, 
                                util.lc_mat_data, util.ilc, util.lc_mat_indices, util.lc_mat_indptr) 
                        
                        # predictor stage
                        do(util.epi2d_dfun,
                                util.dx1, util.dz1, util.x, util.z, util.x_0, util.k_lc, util.lc2, util.k_gc, util.gc2, eps, I)
                        do(util.heun_pred_determ, dt, util.xi, util.x, util.dx1)
                        do(util.heun_pred_determ, dt, util.zi, util.z, util.dz1)
                        
                        # corrector stage
                        do(util.epi2d_dfun,
                                util.dx2, util.dz2, util.xi, util.zi, util.x_0, util.k_lc, util.lc1, util.k_gc, util.gc1, eps, I)
                        do(util.heun_corr_determ, dt, util.x, util.x, util.dx1, util.dx2)
                        do(util.heun_corr_determ, dt, util.z, util.z, util.dz1, util.dz2)
                else :
                        do(util.upbuf_heavi, nv, nsims, np.int32(t), theta, nh, util.xbuf, util.x)
                        # global coupling for heun corrector, i.e. t+1
                        do(util.delays1, 
                                nv, nsims, np.int32(t+1), nh, 
                                util.gc2, util.xbuf, 
                                util.gc_mat_data, util.igc, util.gc_mat_indices, util.gc_mat_indptr,)
                        
                        # local coupling for heun corrector, i.e. t+1
                        do(util.delays1, 
                                nv, nsims, np.int32(t+1), nh, 
                                util.lc2, util.xbuf, 
                                util.lc_mat_data, util.ilc, util.lc_mat_indices, util.lc_mat_indptr) 
                        
                        # predictor stage
                        do(util.epi2d_dfun,
                                util.dx1, util.dz1, util.x, util.z, util.x_0, util.k_lc, util.lc1, util.k_gc, util.gc1, eps, I)
                        do(util.heun_pred_determ, dt, util.xi, util.x, util.dx1)
                        do(util.heun_pred_determ, dt, util.zi, util.z, util.dz1)
                        
                        # corrector stage
                        do(util.epi2d_dfun,
                                util.dx2, util.dz2, util.xi, util.zi, util.x_0, util.k_lc, util.lc2, util.k_gc, util.gc2, eps, I)
                        do(util.heun_corr_determ, dt, util.x, util.x, util.dx1, util.dx2)
                        do(util.heun_corr_determ, dt, util.z, util.z, util.dz1, util.dz2)


        nskip = 10
        niter = 1000
        x_trace = np.zeros((niter//nskip, nv*nsims),'f')
        for i in tqdm.trange(niter):
                step(i)
                if i % nskip == 0:
                        # util.x.get_async(util.queue, x_trace[i//nskip])
                        x_trace[i//nskip] = util.x.get()
        if bs == 32:
                np.savez_compressed("epi2d-sim_batch_gpu_10k_vtx.npz", x_trace=x_trace)


 n sims in parallel : 1
<pyopencl.Context at 0x5592f00fb550 on <pyopencl.Device 'Quadro RTX 4000' on 'NVIDIA CUDA' at 0x5592f0197cf0>>


100%|██████████| 1000/1000 [00:00<00:00, 1385.82it/s]


 n sims in parallel : 2
<pyopencl.Context at 0x5592f432e390 on <pyopencl.Device 'Quadro RTX 4000' on 'NVIDIA CUDA' at 0x5592f0197cf0>>


100%|██████████| 1000/1000 [00:00<00:00, 1353.61it/s]


 n sims in parallel : 4
<pyopencl.Context at 0x5592f4394ac0 on <pyopencl.Device 'Quadro RTX 4000' on 'NVIDIA CUDA' at 0x5592f0197cf0>>


...

100%|██████████| 1000/1000 [00:01<00:00, 692.36it/s]


 n sims in parallel : 128
<pyopencl.Context at 0x5592f2db1f10 on <pyopencl.Device 'Quadro RTX 4000' on 'NVIDIA CUDA' at 0x5592f0197cf0>>


100%|██████████| 1000/1000 [00:02<00:00, 366.21it/s]


 n sims in parallel : 256
<pyopencl.Context at 0x5592f4375770 on <pyopencl.Device 'Quadro RTX 4000' on 'NVIDIA CUDA' at 0x5592f0197cf0>>


100%|██████████| 1000/1000 [00:05<00:00, 179.44it/s]
